<a href="https://colab.research.google.com/github/kimtaewan22/SkeletonDetectLSTM/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_excel("squat1.xlsx")

print(df.head())

   Frame  11_13_15  12_14_16  11_23_25  12_24_26  23_25_27  24_26_28  \
0      1        28        24       123       122       118       110   
1      2        26        26       129       118       118       109   
2      3        27        29       130       115       117       105   
3      4        28        29       127       110       111        98   
4      5        23        26       113       106        94        83   

   14_12_24  13_11_23  14_12_11  13_11_12  32_28_26  31_27_25  26_24_23  \
0        53        46       142       119       121       135       146   
1        53        48       136       127       124       132       147   
2        50        49       133       127       122       130       149   
3        52        47       137       122       119       127       152   
4        54        55       133       134       113       119       161   

   25_23_24  
0       131  
1       127  
2       126  
3       130  
4       139  


In [ ]:
print(df.describe())

            Frame    11_13_15    12_14_16    11_23_25    12_24_26    23_25_27  \
count  523.000000  523.000000  523.000000  523.000000  523.000000  523.000000   
mean   262.000000   25.286807   22.759082  130.835564  124.015296  122.185468   
std    151.121364    6.877831    7.432666   13.876153   21.204436   42.108849   
min      1.000000   12.000000    9.000000   96.000000   75.000000   35.000000   
25%    131.500000   19.000000   15.000000  120.000000  107.000000   78.000000   
50%    262.000000   24.000000   22.000000  137.000000  129.000000  147.000000   
75%    392.500000   31.000000   29.000000  142.000000  143.000000  160.000000   
max    523.000000   44.000000   40.000000  151.000000  153.000000  174.000000   

         24_26_28    14_12_24    13_11_23    14_12_11    13_11_12    32_28_26  \
count  523.000000  523.000000  523.000000  523.000000  523.000000  523.000000   
mean   120.185468   57.196941   52.881453  139.267686  132.110899  132.466539   
std     42.330958    7.0819

In [ ]:
print(df.info())

In [ ]:
import pandas as pd


# 프레임 범위와 열 지정
frame_ranges = [(50, 70), (105, 125),(160, 180), (210, 230), (260, 280), (315,335),(365, 385), (415,435), (465,485)]  # 프레임 범위
columns = ['11_23_25', '12_24_26', '24_26_28', '23_25_27']  # 열

# 각 프레임 범위 내에서 열 값들의 평균 계산
result = []
for frame_range in frame_ranges:
    start_frame, end_frame = frame_range
    selected_rows = df[(df['Frame'] >= start_frame) & (df['Frame'] <= end_frame)]
    mean_values = selected_rows[columns].mean()
    result.append(mean_values)

# 결과 출력
for i, frame_range in enumerate(frame_ranges):
    start_frame, end_frame = frame_range
    print(f"프레임 범위: {start_frame} ~ {end_frame}")
    print(result[i])
    print()

프레임 범위: 50 ~ 70
11_23_25    115.047619
12_24_26    100.523810
24_26_28     68.142857
23_25_27     76.000000
dtype: float64

프레임 범위: 105 ~ 125
11_23_25    117.857143
12_24_26    104.428571
24_26_28     79.523810
23_25_27     83.000000
dtype: float64

프레임 범위: 160 ~ 180
11_23_25    119.095238
12_24_26    102.619048
24_26_28     79.523810
23_25_27     78.285714
dtype: float64

프레임 범위: 210 ~ 230
11_23_25    113.857143
12_24_26    100.285714
24_26_28     73.666667
23_25_27     82.285714
dtype: float64

프레임 범위: 260 ~ 280
11_23_25    115.571429
12_24_26    103.523810
24_26_28     73.904762
23_25_27     66.809524
dtype: float64

프레임 범위: 315 ~ 335
11_23_25    118.333333
12_24_26    100.095238
24_26_28     64.761905
23_25_27     77.095238
dtype: float64

프레임 범위: 365 ~ 385
11_23_25    116.190476
12_24_26    101.857143
24_26_28     71.333333
23_25_27     75.523810
dtype: float64

프레임 범위: 415 ~ 435
11_23_25    115.571429
12_24_26     99.666667
24_26_28     68.619048
23_25_27     70.857143
dtype: flo

#Loss Function - MSE방식

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

selected_columns = ['11_23_25', '12_24_26', '24_26_28', '23_25_27']

# 데이터 불러오기
data = pd.read_excel("squat1.xlsx")

# 데이터 정규화 함수
def normalize_data(data):
    return (data - np.mean(data)) / np.std(data)

# 전체 데이터 정규화
normalized_data = normalize_data(data[selected_columns].values)

# 데이터를 시퀀스로 변환
sequence_length = 10  # 시퀀스 길이 설정
sequences = []
targets = []

for i in range(len(normalized_data) - sequence_length):
    sequence = normalized_data[i:i+sequence_length]
    target = normalized_data[i+sequence_length]
    sequences.append(sequence)
    targets.append(target)

# 데이터를 PyTorch Tensor로 변환
X_tensor = torch.tensor(sequences, dtype=torch.float32)
y_tensor = torch.tensor(targets, dtype=torch.float32)

# LSTM 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        out = self.relu(out)
        return out

# 모델 파라미터 설정
input_size = X_tensor.shape[2]
hidden_size = 128
output_size = X_tensor.shape[2]
num_layers = 2

# 모델 초기화
model = LSTMModel(input_size, hidden_size, output_size, num_layers)

# 손실 함수 및 최적화 알고리즘 설정
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습
num_epochs = 100
batch_size = 32

for epoch in range(num_epochs):
    permutation = torch.randperm(X_tensor.size(0))
    for i in range(0, X_tensor.size(0), batch_size):
        indices = permutation[i:i+batch_size]
        batch_X = X_tensor[indices]
        batch_y = y_tensor[indices]
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch: {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# 예측
test_sequence = normalized_data[-sequence_length:]  # 마지막 시퀀스 추출
test_input = torch.tensor(test_sequence, dtype=torch.float32).unsqueeze(0)
predicted_output = model(test_input)

# 예측 결과 디노멜라이즈
predicted_output_denorm = predicted_output.squeeze().detach().numpy() * np.std(data[selected_columns].values) + np.mean(data[selected_columns].values)

# 예측 결과 출력
print("예측 결과:")
print(predicted_output_denorm)


Epoch: 10/100, Loss: 0.01126687228679657
Epoch: 20/100, Loss: 0.012224841862916946
Epoch: 30/100, Loss: 0.006596724037081003
Epoch: 40/100, Loss: 0.005257095210254192
Epoch: 50/100, Loss: 0.009997071698307991
Epoch: 60/100, Loss: 0.011506931856274605
Epoch: 70/100, Loss: 0.0099712572991848
Epoch: 80/100, Loss: 0.0011757557513192296
Epoch: 90/100, Loss: 0.010809794999659061
Epoch: 100/100, Loss: 0.0063133444637060165
예측 결과:
[142.34468 144.98401 161.67628 159.8289 ]


In [ ]:
from sklearn.metrics import mean_squared_error

# 실제값
actual_value_last = actual_values[-1]

next_actual_data = data.loc[-1:-1, selected_columns],

# 예측값 디노멜라이즈
predicted_output_denorm = predicted_output.squeeze().detach().numpy() * np.std(data[selected_columns].values) + np.mean(data[selected_columns].values)

print("예측값",predicted_output_denorm)
print("실측값", actual_value_last)

# RMSE 계산
rmse = np.sqrt(mean_squared_error([actual_value_last], [predicted_output_denorm]))

print("RMSE:", rmse)

예측값 [142.34468 144.98401 161.67628 159.8289 ]
실측값 [143 147 161 159]
RMSE: 1.1872347560544905


In [ ]:
from sklearn.metrics import r2_score

# R^2 계산
r2 = r2_score(actual_value_last, predicted_output_denorm)

print("R^2:", r2)

R^2: 0.9601286818580503


# Loss Function - Huber Loss 방식


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

selected_columns = ['11_23_25', '12_24_26', '24_26_28', '23_25_27']

# 데이터 불러오기
data = pd.read_excel("squat1.xlsx")

# 데이터 정규화 함수
def normalize_data(data):
    return (data - np.mean(data)) / np.std(data)

# 전체 데이터 정규화
normalized_data = normalize_data(data[selected_columns].values)

# 데이터를 시퀀스로 변환
sequence_length = 10  # 시퀀스 길이 설정
sequences = []
targets = []

for i in range(len(normalized_data) - sequence_length):
    sequence = normalized_data[i:i+sequence_length]
    target = normalized_data[i+sequence_length]
    sequences.append(sequence)
    targets.append(target)

# 데이터를 PyTorch Tensor로 변환
X_tensor = torch.tensor(sequences, dtype=torch.float32)
y_tensor = torch.tensor(targets, dtype=torch.float32)

# LSTM 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        out = self.relu(out)
        return out

# 모델 파라미터 설정
input_size = X_tensor.shape[2]
hidden_size = 128
output_size = X_tensor.shape[2]
num_layers = 2

# 모델 초기화
model = LSTMModel(input_size, hidden_size, output_size, num_layers)

# 손실 함수 및 최적화 알고리즘 설정
criterion = nn.SmoothL1Loss()  # Huber Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습
num_epochs = 100
batch_size = 32

for epoch in range(num_epochs):
    permutation = torch.randperm(X_tensor.size(0))
    for i in range(0, X_tensor.size(0), batch_size):
        indices = permutation[i:i+batch_size]
        batch_X = X_tensor[indices]
        batch_y = y_tensor[indices]
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch: {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# 예측
test_sequence = normalized_data[-sequence_length:]  # 마지막 시퀀스 추출
test_input = torch.tensor(test_sequence, dtype=torch.float32).unsqueeze(0)
predicted_output = model(test_input)

# 예측 결과 디노멜라이즈
predicted_output_denorm = predicted_output.squeeze().detach().numpy() * np.std(data[selected_columns].values) + np.mean(data[selected_columns].values)

# 예측 결과 출력
print("예측 결과:")
print(predicted_output_denorm)


Epoch: 10/100, Loss: 0.754517674446106
Epoch: 20/100, Loss: 0.07914291322231293
Epoch: 30/100, Loss: 0.009988589212298393
Epoch: 40/100, Loss: 0.4441593289375305
Epoch: 50/100, Loss: 0.9571226835250854
Epoch: 60/100, Loss: 0.013068306259810925
Epoch: 70/100, Loss: 0.040757790207862854
Epoch: 80/100, Loss: 0.839005708694458
Epoch: 90/100, Loss: 0.0024806614965200424
Epoch: 100/100, Loss: 0.3039771318435669
예측 결과:
[142.79924 145.35071 160.87032 161.56758]


In [ ]:
from sklearn.metrics import mean_squared_error

# 실제값
actual_value_last = actual_values[-1]

next_actual_data = data.loc[-1:-1, selected_columns],

# 예측값 디노멜라이즈
predicted_output_denorm = predicted_output.squeeze().detach().numpy() * np.std(data[selected_columns].values) + np.mean(data[selected_columns].values)

print("예측값",predicted_output_denorm)
print("실측값", actual_value_last)

# RMSE 계산
rmse = np.sqrt(mean_squared_error([actual_value_last], [predicted_output_denorm]))

print("RMSE:", rmse)

예측값 [142.79924 145.35071 160.87032 161.56758]
실측값 [143 147 161 159]
RMSE: 1.5305031659031418
